In [3]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import numpy as np
import pickle
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

In [4]:
%matplotlib widget

In [5]:
with open("features.pkl", "rb") as f:
    df_features = pickle.load(f)
nd_features = df_features.values

nd_features.shape

(18580, 6)

In [6]:
# read iterations
with open("output/20240420_184538/20240420_184538_analysis_population.pkl", "rb") as f:
    df_iterations = pickle.load(f)

df_iterations.describe()

,p1,p2,p3,p4,p5,p6,score,pop,iteration,path_id
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.0,100000.000,100000.000,100000.0
mean,0.128438,0.544384,0.220869,0.586252,0.280568,0.039520,-1.0,251.500,249.500,0.0
std,0.074738,0.111038,0.099093,0.090135,0.103557,0.062496,0.0,144.338,144.338,0.0
min,0.002194,0.069219,0.000993,0.151914,0.011036,0.000005,-1.0,2.000,0.000,0.0
25%,0.072999,0.462637,0.145595,0.527899,0.207128,0.006411,-1.0,126.750,124.750,0.0
50%,0.110355,0.547562,0.213000,0.586466,0.257567,0.016881,-1.0,251.500,249.500,0.0
75%,0.171743,0.620106,0.291473,0.652050,0.332844,0.039877,-1.0,376.250,374.250,0.0
max,0.567949,0.878750,0.593685,0.900997,0.817247,0.815931,-1.0,501.000,499.000,0.0


In [ ]:
######## comment out

for iteration in df_iterations.iteration.unique()[:1]:
    df_epoch = df_iterations[df_iterations.iteration == iteration]
    df_epoch = df_epoch[["p1", "p2", "p3", "p4", "p5", "p6"]]
    
df_epoch.describe()

In [7]:
# Scale the features
# scaler = StandardScaler()
# data_scaled = scaler.fit_transform(nd_features)

# Apply K-means clustering
kmeans = KMeans(n_clusters=5, random_state=42).fit(nd_features)
labels = kmeans.labels_

# Save the DataFrame with cluster labels
df_features["cluster"] = labels
with open("features_clustered_TEST.pkl", "wb") as f:
    pickle.dump(df_features, f)
###

# print(df_features.head())
# print(df_features.describe())
print(df_features.cluster.value_counts())

c:\Users\emin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


cluster
4    5553
0    4711
2    3965
1    3591
3     760
Name: count, dtype: int64


In [ ]:
#### comment out

# find visited indexes
visited_indexes = []
for individual in df_epoch.values:
    for index, feature in enumerate(nd_features):
        if np.array_equal(individual, feature):
            visited_indexes.append(index)
            break
visited_indexes = np.array(visited_indexes)

visited_indexes

In [18]:
%matplotlib widget

In [ ]:
# VISUALIZE THE CLUSTERS (2D) with t-SNE
tsne = TSNE(n_components=2, random_state=0)
features_data_tsne = tsne.fit_transform(nd_features)
df_tsne = pd.DataFrame(features_data_tsne, columns=["x", "y"])

for i, iteration in enumerate(df_iterations.iteration.unique()):
    df_epoch = df_iterations[df_iterations.iteration == iteration]
    df_epoch = df_epoch[["p1", "p2", "p3", "p4", "p5", "p6"]]
    
    # find visited indexes
    visited_indexes = []
    for individual in df_epoch.values:
        for index, feature in enumerate(nd_features):
            if np.array_equal(individual, feature):
                visited_indexes.append(index)
                break
    visited_indexes = np.array(visited_indexes)

    df_tsne["cluster"] = labels

    plt.clf()
    plt.figure(figsize=(10, 10))
    plt.scatter(df_tsne["x"], df_tsne["y"], s=1, c=df_tsne["cluster"], cmap="viridis")
    plt.scatter(df_tsne["x"].iloc[visited_indexes], df_tsne["y"].iloc[visited_indexes], c="red", s=3, marker="^")
    plt.show()
    plt.savefig(f"clusters_TEST_{i}.png")

In [ ]:
# VISUALIZE THE CLUSTERS (3D) with t-SNE
tsne = TSNE(n_components=3, random_state=0)
features_data_tsne = tsne.fit_transform(nd_features)

df_tsne = pd.DataFrame(features_data_tsne, columns=["x", "y", "z"])
df_tsne["cluster"] = labels

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection="3d")
ax.scatter(
    df_tsne["x"], df_tsne["y"], df_tsne["z"], c=df_tsne["cluster"], cmap="viridis"
)
plt.show()
plt.savefig("clusters_3d_TEST.png")
